In [1]:
import tensorflow as tf
import h5py
import sys

sys.path.append("../Data_Preprocessing/")
from Preprocess_Utilities import *
from Metric import *

In [ ]:
# restore trained model & training info into session
sess = tf.InteractiveSession()

saver = tf.train.import_meta_graph('./Result/models/res_serv_road_livi/res_serv_road_livi.meta')
saver.restore(sess, tf.train.latest_checkpoint('./Result/models/res_serv_road_livi/'))

In [8]:
training_info = h5py.File('./Result/models/res_serv_road_livi/training_info.h5')
index = training_info['Index']
norm  = training_info['Norm']

train_index = np.array(index['train_index'])
test_index  = index['test_index']
mu    = norm['mu']
sigma = norm['sigma']
training_info.close()

In [10]:
train_index.shape

(64,)

In [4]:
# read in & re-construct training/test set
data_path = '../Data/090085/Road_Data/res_serv_road_livi/patch_set.h5'
data = h5py.File(data_path, 'r')
X = np.array(data['image_patch'])
Y = np.array(data['road_existence'])
Road_patch = np.array(data['road_patch'])
data.close()

train_x = X[train_index].flatten().reshape((train_index.size, -1))
train_y = Y[train_index]
train_road_patch = Road_patch[train_index]

test_x = X[test_index].flatten().reshape((test_index.size, -1))
test_y = Y[test_index]
test_road_patch = Road_patch[test_index]

print(train_x.shape, train_y.shape, train_road_patch.shape)
print(test_x.shape, test_y.shape, test_road_patch.shape)

(64, 5488) (64,) (64, 28, 28)
(86833, 5488) (86833,) (86833, 28, 28)


In [12]:
# predict training & test set

# training set
train_metric = Metric()
train_pred_label = np.zeros(train_index.size)

batch_size = 64
batch_num = int(train_index.size/batch_size)+1
for i in range(batch_num):
    start = i%batch_num * batch_size
    end = start + batch_size

    if end > train_index.size:
        end = train_index.size
    
    batch = [((train_x[start:end]-mu)/sigma), np.matrix(train_y[start:end]).T]
 
    # record metric
    pred = prediction.eval(feed_dict={x:batch[0]})
    train_metric.accumulate(pred, batch[1])

print(train_metric.cal_metric())

# test set
test_metric = Metric()
test_pred_label = np.zeros(test_index.size)

batch_size = 64
batch_num = int(test_index.size/batch_size)+1
for i in range(batch_num):
    start = i%batch_num * batch_size
    end = start + batch_size

    if end > test_index.size:
        end = test_index.size
    
    batch = [((test_x[start:end]-mu)/sigma), np.matrix(test_y[start:end]).T]

    # record metric   
    pred = prediction.eval(feed_dict={x:batch[0]})
    test_metric.accumulate(pred, batch[1])

print(test_metric.cal_metric())

NameError: name 'mu' is not defined